# Working on these

- [ ] Benchmark from OpenAI x COVID-19-deepset with BERTScore
- [ ] Benchmark from petal.org
- [ ] Evaluate extractive and abstracting summary
- [ ] Mis-information safety mechanism
- [ ] Traceability (paragraphs)
- [ ] Traceability (words)

## Test set creation

- From `covid_qa_deepset`
- 8-2 split, rng = 2023, test n = 404

In [ ]:
from askem.data import COVID_QA
from askem.openai import get_answer, count_token, split_context

test_data = COVID_QA.train_test_split(test_size=0.2, seed=2023)["test"]
test_data

## OpenAI

In [ ]:
from peewee import *
from tqdm.notebook import tqdm

db = SqliteDatabase("my_database.db")


class GPTBench(Model):
    id = IntegerField(primary_key=True)
    context = TextField()
    question = TextField()
    true_answer = TextField()
    gpt_answer = TextField()

    class Meta:
        database = db


# Connect to the database and create the table
db.connect()
db.create_tables([GPTBench])

In [ ]:
for id, context, question, true_answer in tqdm(
    zip(
        test_data["id"],
        test_data["context"],
        test_data["question"],
        test_data["answers"],
    )
):
    gpt_answer = get_answer(context, question)
    GPTBench.insert(
        id=id,
        context=context,
        question=question,
        true_answer=true_answer,
        gpt_answer=gpt_answer,
    ).on_conflict_ignore().execute()